# NOTEBOOK 3.3 Spark SQL



In [1]:
from pyspark.sql import SparkSession
# from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.getOrCreate()

25/06/12 15:50:29 WARN Utils: Your hostname, PC25. resolves to a loopback address: 127.0.1.1; using 192.168.76.195 instead (on interface eth0)
25/06/12 15:50:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/12 15:50:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1. Read from CSV file with DDL string as schema

In [2]:
schema = "ID INT, Employee STRING, Department STRING, Salary DOUBLE, Age INT"
salary_df = spark.read.csv('data/salary_data2.csv', header=True, schema=schema)

salary_df.show()

+---+--------+----------+------+---+
| ID|Employee|Department|Salary|Age|
+---+--------+----------+------+---+
|  1|    John| Field-eng|3500.0| 40|
|  7|  Martin|   Finance|3500.0| 26|
|  3|   Maria|   Finance|3500.0| 28|
|  4| Michael|     Sales|3000.0| 20|
|  5|   Kelly|   Finance|3500.0| 35|
|  2|  Robert|     Sales|4000.0| 38|
|  6|    Kate|   Finance|3000.0| 45|
|  8|   Kiran|     Sales|2200.0| 35|
+---+--------+----------+------+---+



In [3]:
# Perform transformations on the loaded data
filtered_df = salary_df.filter(salary_df["Salary"] > 3000)
filtered_df.show()

+---+--------+----------+------+---+
| ID|Employee|Department|Salary|Age|
+---+--------+----------+------+---+
|  1|    John| Field-eng|3500.0| 40|
|  7|  Martin|   Finance|3500.0| 26|
|  3|   Maria|   Finance|3500.0| 28|
|  5|   Kelly|   Finance|3500.0| 35|
|  2|  Robert|     Sales|4000.0| 38|
+---+--------+----------+------+---+



## 2. Temporary Views
Temporary views enables developers us run SQL queries in a program and get the result as a DataFrame. There are 2 types of temporary views:
- **Local Temporary View**
    - The default temporary view.
    - Lasts for the session in which they are created. 
- **Global Temporary View**
    - If we want to have views available across various sessions, we need to create Global Temporary Views. The view definition is stored in the default database, **global_temp**. Once a view is created, we need to use the fully qualified name to access it in a query.

### 2.1 Create (Local) Temporary View

In [4]:
# Save the processed data as a table
filtered_df.createOrReplaceTempView("high_salary_employees")

# Perform SQL queries on the saved table
results_df = spark.sql("SELECT * FROM high_salary_employees ")
results_df.show()

+---+--------+----------+------+---+
| ID|Employee|Department|Salary|Age|
+---+--------+----------+------+---+
|  1|    John| Field-eng|3500.0| 40|
|  7|  Martin|   Finance|3500.0| 26|
|  3|   Maria|   Finance|3500.0| 28|
|  5|   Kelly|   Finance|3500.0| 35|
|  2|  Robert|     Sales|4000.0| 38|
+---+--------+----------+------+---+



### 2.2 Create Global Temporary View

In [5]:
salary_df.createGlobalTempView("salaries")

print("Data from current spark session")
spark.sql("SELECT * FROM global_temp.salaries").show()

# Global temporary view is tied to a system database `global_temp`
print("Data from a new spark session")
spark.newSession().sql("SELECT * FROM global_temp.salaries").show()

Data from current spark session
+---+--------+----------+------+---+
| ID|Employee|Department|Salary|Age|
+---+--------+----------+------+---+
|  1|    John| Field-eng|3500.0| 40|
|  7|  Martin|   Finance|3500.0| 26|
|  3|   Maria|   Finance|3500.0| 28|
|  4| Michael|     Sales|3000.0| 20|
|  5|   Kelly|   Finance|3500.0| 35|
|  2|  Robert|     Sales|4000.0| 38|
|  6|    Kate|   Finance|3000.0| 45|
|  8|   Kiran|     Sales|2200.0| 35|
+---+--------+----------+------+---+

Data from a new spark session
+---+--------+----------+------+---+
| ID|Employee|Department|Salary|Age|
+---+--------+----------+------+---+
|  1|    John| Field-eng|3500.0| 40|
|  7|  Martin|   Finance|3500.0| 26|
|  3|   Maria|   Finance|3500.0| 28|
|  4| Michael|     Sales|3000.0| 20|
|  5|   Kelly|   Finance|3500.0| 35|
|  2|  Robert|     Sales|4000.0| 38|
|  6|    Kate|   Finance|3000.0| 45|
|  8|   Kiran|     Sales|2200.0| 35|
+---+--------+----------+------+---+



## 3. Running SQL Queries Programmatically

In [6]:
salary_df.createOrReplaceTempView("employees")

# Select columns
sql_string = "SELECT Employee, Department, Salary, Age FROM employees WHERE age > 30"
results_df = spark.sql(sql_string)
results_df.show()

+--------+----------+------+---+
|Employee|Department|Salary|Age|
+--------+----------+------+---+
|    John| Field-eng|3500.0| 40|
|   Kelly|   Finance|3500.0| 35|
|  Robert|     Sales|4000.0| 38|
|    Kate|   Finance|3000.0| 45|
|   Kiran|     Sales|2200.0| 35|
+--------+----------+------+---+



In [7]:
# Perform an aggregation to calculate the average salary
sql_string = "SELECT AVG(Salary) AS average_salary FROM employees"
average_salary_df = spark.sql(sql_string)
average_salary_df.show()

+--------------+
|average_salary|
+--------------+
|        3275.0|
+--------------+



In [8]:
# Sort the data based on the salary column in descending order
sql_string = "SELECT * FROM employees ORDER BY Salary DESC"
sorted_df = spark.sql(sql_string)
sorted_df.show()

+---+--------+----------+------+---+
| ID|Employee|Department|Salary|Age|
+---+--------+----------+------+---+
|  2|  Robert|     Sales|4000.0| 38|
|  3|   Maria|   Finance|3500.0| 28|
|  5|   Kelly|   Finance|3500.0| 35|
|  7|  Martin|   Finance|3500.0| 26|
|  1|    John| Field-eng|3500.0| 40|
|  6|    Kate|   Finance|3000.0| 45|
|  4| Michael|     Sales|3000.0| 20|
|  8|   Kiran|     Sales|2200.0| 35|
+---+--------+----------+------+---+



In [9]:
# Sort the data based on the salary column in descending order
sql_string = """SELECT Employee, Department, Salary, Age FROM employees
WHERE age > 30 AND Salary > 3000 ORDER BY Salary DESC"""

filtered_df = spark.sql(sql_string)
filtered_df.show()

+--------+----------+------+---+
|Employee|Department|Salary|Age|
+--------+----------+------+---+
|  Robert|     Sales|4000.0| 38|
|    John| Field-eng|3500.0| 40|
|   Kelly|   Finance|3500.0| 35|
+--------+----------+------+---+



In [10]:
# Group the data based on the Department column and take average salary for each department
sql_string = "SELECT Department, avg(Salary) FROM employees GROUP BY Department"
grouped_df = spark.sql(sql_string)
grouped_df.show()

+----------+------------------+
|Department|       avg(Salary)|
+----------+------------------+
| Field-eng|            3500.0|
|   Finance|            3375.0|
|     Sales|3066.6666666666665|
+----------+------------------+



In [11]:
# Perform grouping and multiple aggregations
sql_string = "SELECT Department, sum(Salary) AS total_salary, max(Salary) AS max_salary FROM employees GROUP BY Department"
aggregated_df = spark.sql(sql_string)
aggregated_df.show()

+----------+------------+----------+
|Department|total_salary|max_salary|
+----------+------------+----------+
| Field-eng|      3500.0|    3500.0|
|   Finance|     13500.0|    3500.0|
|     Sales|      9200.0|    4000.0|
+----------+------------+----------+



In [12]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, sum

# Define the window specification
window_spec = Window.partitionBy("Department").orderBy("Age")

# Calculate the cumulative sum using window function
cumulative_sum_df = salary_df.withColumn("cumulative_sum", sum(col("Salary")).over(window_spec))
cumulative_sum_df.show()

+---+--------+----------+------+---+--------------+
| ID|Employee|Department|Salary|Age|cumulative_sum|
+---+--------+----------+------+---+--------------+
|  1|    John| Field-eng|3500.0| 40|        3500.0|
|  7|  Martin|   Finance|3500.0| 26|        3500.0|
|  3|   Maria|   Finance|3500.0| 28|        7000.0|
|  5|   Kelly|   Finance|3500.0| 35|       10500.0|
|  6|    Kate|   Finance|3000.0| 45|       13500.0|
|  4| Michael|     Sales|3000.0| 20|        3000.0|
|  8|   Kiran|     Sales|2200.0| 35|        5200.0|
|  2|  Robert|     Sales|4000.0| 38|        9200.0|
+---+--------+----------+------+---+--------------+



## 4. More Examples on Spark SQL

In [13]:
sales_df = spark.read.option("sep", "\t")\
    .option("header", "true")\
    .csv("data/sales.csv")

sales_df.show()
sales_df.printSchema()

+----+-----------+----------+--------+
|code|description|unit_price|quantity|
+----+-----------+----------+--------+
|1005|        pen|       2.5|       4|
|1007|     pencil|       1.0|      10|
|1001|   notebook|       5.0|       2|
|1003|      ruler|       1.0|       1|
|1002| calculator|      55.0|       1|
+----+-----------+----------+--------+

root
 |-- code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- unit_price: string (nullable = true)
 |-- quantity: string (nullable = true)



### 4.1 Use SparkSQL to read the columns with correct data types

In [14]:
sales_df.createOrReplaceTempView('sales')
sales_df = spark.sql("SELECT code, description, DOUBLE(unit_price), INT(quantity) from sales")
sales_df.printSchema()
sales_df.show()

root
 |-- code: string (nullable = true)
 |-- description: string (nullable = true)
 |-- unit_price: double (nullable = true)
 |-- quantity: integer (nullable = true)

+----+-----------+----------+--------+
|code|description|unit_price|quantity|
+----+-----------+----------+--------+
|1005|        pen|       2.5|       4|
|1007|     pencil|       1.0|      10|
|1001|   notebook|       5.0|       2|
|1003|      ruler|       1.0|       1|
|1002| calculator|      55.0|       1|
+----+-----------+----------+--------+



In [15]:
df = spark.read.json("data/nyt2.json")
df.show(2)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+---------+-------------+
|                 _id|  amazon_product_url|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|    title|weeks_on_list|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+---------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {NULL, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, NULL}|{{1212883200000}}|Little, Brown| {2}|           {1}| THE HOST|          {3}|
+--------------------+--------------------+------------

In [16]:
# Registering the table
df.createOrReplaceTempView("books")

# Select all the rows from book_df and display the first 3 rows
spark.sql("select * from books").show(3)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {NULL, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, NULL}|{{1212883200000}}|Little, Brown| {2}|           {1}|            THE HOST|          {3}|


In [17]:
# Summarize the number of books by themes
spark.sql("select \
CASE WHEN description LIKE '%love%' THEN 'Love_Theme' \
WHEN description LIKE '%hate%' THEN 'Hate_Theme' \
WHEN description LIKE '%happy%' THEN 'Happiness_Theme' \
WHEN description LIKE '%anger%' THEN 'Anger_Theme' \
WHEN description LIKE '%horror%' THEN 'Horror_Theme' \
WHEN description LIKE '%death%' THEN 'Criminal_Theme' \
WHEN description LIKE '%detective%' THEN 'Mystery_Theme' \
ELSE 'Other_Themes' \
END Themes \
from books").groupBy('Themes').count().show()

+---------------+-----+
|         Themes|count|
+---------------+-----+
|    Anger_Theme|  203|
|   Other_Themes| 8778|
|  Mystery_Theme|  454|
|     Hate_Theme|   23|
|     Love_Theme|  392|
|Happiness_Theme|   34|
|   Horror_Theme|    6|
| Criminal_Theme|  305|
+---------------+-----+



## 5. Caching, Checkpointing, Repartitioning and Coalesing
https://spark.apache.org/docs/1.1.1/api/python/pyspark.rdd.RDD-class.html

### 5.1 Caching
- Running the cache() transformation on a Spark DataFrame stores the DataFrame in memory across the cluster for faster access during ubsequent actions.
- In-Memory Storage: Spark stores the partitions of the DataFrame in memory. If there isn’t enough memory, some partitions may be recomputed as needed.
    - Performance Gain: Subsequent actions on the cached DataFrame are faster because Spark reuses the in-memory data instead of recomputing it.
    - Cluster-Wide Effect: Caching applies across the cluster, meaning each node tries to keep its share of data in memory.

#### 5.1(a) cache()

In [18]:
# Cache a DataFrame (defaults to MEMORY_AND_DISK)
df.cache()  

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>]

#### 5.1(b) persist()
Used to specify the storage level.

**Storage Levels**:
- StorageLevel.MEMORY_ONLY
- StorageLevel.MEMORY_AND_DISK
- StorageLevel.DISK_ONLY

In [19]:
from pyspark import StorageLevel

# Cache a DataFrame; df.cache() (i.e., without parameters) defaults to MEMORY_AND_DISK
df.persist(StorageLevel.MEMORY_ONLY)  

25/06/12 15:50:39 WARN CacheManager: Asked to cache already cached data.


DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>]

#### 5.1(c) unpersist()

Removes a DataFrame or RDD from memory and/or disk where it was cached or persisted. When you no longer need a cached/persisted dataset, calling unpersist() helps free up resources (memory and/or disk), making room for other computations.

In [20]:
df.unpersist()

DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>]

### 5.2 checkpoint()
Steps:
1. Set a checkpointing directory using setCheckpointDir() method.
2. Store the content of baseRDD using checkpoint().

In [21]:
sc = spark.sparkContext
sc.setCheckpointDir("checkpoints/checkpointing")

# checkpointing returns a new DataFrame
df_cp = df.checkpoint()

# trigger it
df_cp.count()

# use the checkpointed DataFrame
df_cp.show(3)

+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|                 _id|  amazon_product_url|         author| bestsellers_date|         description|        price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|
+--------------------+--------------------+---------------+-----------------+--------------------+-------------+-----------------+-------------+----+--------------+--------------------+-------------+
|{5b4aa4ead3089013...|http://www.amazon...|  Dean R Koontz|{{1211587200000}}|Odd Thomas, who c...|   {NULL, 27}|{{1212883200000}}|       Bantam| {1}|           {0}|           ODD HOURS|          {1}|
|{5b4aa4ead3089013...|http://www.amazon...|Stephenie Meyer|{{1211587200000}}|Aliens have taken...|{25.99, NULL}|{{1212883200000}}|Little, Brown| {2}|           {1}|            THE HOST|          {3}|


### 5.3 repartition()

In [22]:
df.rdd.getNumPartitions()

2

In [23]:
# Increase the number of partitions to 10
df.repartition(10).rdd.getNumPartitions()

10

### 5.4 coalesce(1)

In [24]:
# Reduce the number of partitions to 1 partition
df.coalesce(4).rdd.getNumPartitions()

2

## 6. Convert Spark DataFrame to RDD

In [25]:
# Convert the dataframe into an RDD
books_rdd = df.rdd

print("type(books_rdd): ", type(books_rdd))
books_rdd.take(2)

type(books_rdd):  <class 'pyspark.rdd.RDD'>


[Row(_id=Row($oid='5b4aa4ead3089013507db18b'), amazon_product_url='http://www.amazon.com/Odd-Hours-Dean-Koontz/dp/0553807056?tag=NYTBS-20', author='Dean R Koontz', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Odd Thomas, who can communicate with the dead, confronts evil forces in a California coastal town.', price=Row($numberDouble=None, $numberInt='27'), published_date=Row($date=Row($numberLong='1212883200000')), publisher='Bantam', rank=Row($numberInt='1'), rank_last_week=Row($numberInt='0'), title='ODD HOURS', weeks_on_list=Row($numberInt='1')),
 Row(_id=Row($oid='5b4aa4ead3089013507db18c'), amazon_product_url='http://www.amazon.com/The-Host-Novel-Stephenie-Meyer/dp/0316218502?tag=NYTBS-20', author='Stephenie Meyer', bestsellers_date=Row($date=Row($numberLong='1211587200000')), description='Aliens have taken control of the minds and bodies of most humans, but one woman won’t surrender.', price=Row($numberDouble='25.99', $numberInt=None), published_date=

In [28]:
# Query with Catalyst Optimizer for CSV data
result_df = salary_df.select("employee", "department").filter(salary_df["Salary"] > 3500)

# Explain the optimized query plan
print("\nPhysical and logical dataframe plans")
result_df.explain()


Physical and logical dataframe plans
== Physical Plan ==
*(1) Project [employee#1, department#2]
+- *(1) Filter (isnotnull(Salary#3) AND (Salary#3 > 3500.0))
   +- FileScan csv [Employee#1,Department#2,Salary#3] Batched: false, DataFilters: [isnotnull(Salary#3), (Salary#3 > 3500.0)], Format: CSV, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/user/student/data/salary_data2.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Salary), GreaterThan(Salary,3500.0)], ReadSchema: struct<Employee:string,Department:string,Salary:double>




In [29]:
salary_df.write.parquet("data/salary.parquet")

parquet_df = spark.read.parquet('data/salary.parquet')
parquet_df.show()

25/06/12 16:00:46 WARN MemoryManager: Total allocation exceeds 50.00% (508,559,360 bytes) of heap memory
Scaling row group sizes to 94.73% for 4 writers
25/06/12 16:00:46 WARN MemoryManager: Total allocation exceeds 50.00% (508,559,360 bytes) of heap memory
Scaling row group sizes to 75.78% for 5 writers
25/06/12 16:00:46 WARN MemoryManager: Total allocation exceeds 50.00% (508,559,360 bytes) of heap memory
Scaling row group sizes to 63.15% for 6 writers
25/06/12 16:00:46 WARN MemoryManager: Total allocation exceeds 50.00% (508,559,360 bytes) of heap memory
Scaling row group sizes to 54.13% for 7 writers
25/06/12 16:00:46 WARN MemoryManager: Total allocation exceeds 50.00% (508,559,360 bytes) of heap memory
Scaling row group sizes to 47.36% for 8 writers
25/06/12 16:00:46 WARN MemoryManager: Total allocation exceeds 50.00% (508,559,360 bytes) of heap memory
Scaling row group sizes to 54.13% for 7 writers
25/06/12 16:00:46 WARN MemoryManager: Total allocation exceeds 50.00% (508,559,360

+---+--------+----------+------+---+
| ID|Employee|Department|Salary|Age|
+---+--------+----------+------+---+
|  1|    John| Field-eng|3500.0| 40|
|  7|  Martin|   Finance|3500.0| 26|
|  4| Michael|     Sales|3000.0| 20|
|  5|   Kelly|   Finance|3500.0| 35|
|  3|   Maria|   Finance|3500.0| 28|
|  2|  Robert|     Sales|4000.0| 38|
|  6|    Kate|   Finance|3000.0| 45|
|  8|   Kiran|     Sales|2200.0| 35|
+---+--------+----------+------+---+



In [30]:
# Query with Catalyst Optimizer for CSV data
parquet_result_df = parquet_df.select("employee", "department").filter(parquet_df["Salary"] > 3500)

# Explain the optimized query plan
print("\nPhysical and logical dataframe plans")
parquet_result_df.explain()


Physical and logical dataframe plans
== Physical Plan ==
*(1) Project [employee#807, department#808]
+- *(1) Filter (isnotnull(Salary#809) AND (Salary#809 > 3500.0))
   +- *(1) ColumnarToRow
      +- FileScan parquet [Employee#807,Department#808,Salary#809] Batched: true, DataFilters: [isnotnull(Salary#809), (Salary#809 > 3500.0)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[hdfs://localhost:9000/user/student/data/salary.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(Salary), GreaterThan(Salary,3500.0)], ReadSchema: struct<Employee:string,Department:string,Salary:double>




In [13]:
spark.stop()